In [64]:
import networkx as nx
import random
import numpy as np

In [65]:
#data
G = nx.read_graphml('test.graphml') #graph

#fold-change dict
fold_change = {}
k = list(range(0,5))
for i in G.nodes():
    fold_change[i] = random.choice(k)        

In [119]:
def overlay(graph, fold_change_dict):
    '''Overlaying the fold-change data onto the graph.'''
    threshold = 2 #arbitary for test dict
    for i in graph.nodes():
        if fold_change_dict[i] > threshold: #increased expression
            graph.add_node(i, change = 1)
        elif fold_change_dict[i] < threshold: #decreased expression
            graph.add_node(i, change = -1)
        else: #no change 
            graph.add_node(i, change = 0)
    return graph

def shortest_path(graph,hyp_node):
    '''Returning the shorstest path of the hype nodes with all the other nodes.'''
    return nx.single_source_shortest_path(graph,hyp_node)

def edge_label_value(graph,path_list):
    '''Return the product of the edges value of the path.'''
    if len(path_list) == 1:
        return 0
    else:
        edge = {'increase' : 1, 'decrease' : -1}
        edge_list = []
        
        for i in range(len(path_list) - 1):
            k = graph.edges[path_list[i], path_list[i+1]] #edge dictionary attribute
            edge_list.append(edge[k['Relation']])
        return np.prod(edge_list)

    
def node_label_value(graph,path_list):
    '''Returns the product of the starting and end node of the path.'''
    if len(path_list) == 1:
        return 0
    else:
        node_list = []
        for i,j in graph.nodes.data():
            if i == path_list[0] or i == path_list[-1]:
                node_list.append(j['change'])
        return np.prod(node_list)
    

def calculate_concordance(graph,hyp_node):
    if hyp_node not in graph:
        raise ValueError('Node not preset in graph.')
    else:
        concordance_count = 0
        non_concordance_count = 0
        
        path_dict = shortest_path(graph,hyp_node)
        for i in path_dict.keys():
            path = path_dict[i] #path to travel
            edge_val = edge_label_value(graph,path) #edge product value
            node_val = node_label_value(graph,path) # node product value
            if edge_val == node_val:
                concordance_count += 1
            else:
                non_concordance_count += 1
    return (hyp_node,concordance_count,non_concordance_count)

In [120]:
overlay_graph = overlay(G, fold_change)
calculate_concordance(overlay_graph, 'IFNG')   

('IFNG', 10, 24)